In [59]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [60]:
#!pip install tensorflow==2.5.0

In [61]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [62]:
import tensorflow as tf
import os
import zipfile

!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
100% 2.67M/2.67M [00:00<00:00, 41.8MB/s]


In [63]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # please do not remove # model fitting inline comments.
    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
      # stop training the model after reaching 99% accuracy
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >=0.999):
          print("\n Reached 99.9% accuracy, stopping...")
          self.model.stop_training = True
        
        
    callbacks = myCallback()
    
    # this Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
      # define 3 convolutional layers
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape= (150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      # flatten the data
      tf.keras.layers.Flatten(),
      # hidden layer with 512 nodes
      tf.keras.layers.Dense(512, activation='relu'),
      # output layer
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy', 
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy']
                 )
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        './h-or-s',
        target_size=(150, 150),
        batch_size=64,
        class_mode='binary'
    )
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(
        train_generator,
        steps_per_epoch=1,
        epochs=15,
        verbose=1,
        callbacks=[callbacks]
    )

    
    return history.history['accuracy'][-1]

In [64]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
1/1 [==============================] - 1s 1s/step - loss: 0.6988 - accuracy: 0.5469
Epoch 2/15
1/1 [==============================] - 0s 193ms/step - loss: 3.9627 - accuracy: 0.5000
Epoch 3/15
1/1 [==============================] - 0s 181ms/step - loss: 2.2135 - accuracy: 0.5156
Epoch 4/15
1/1 [==============================] - 0s 71ms/step - loss: 0.7596 - accuracy: 0.3750
Epoch 5/15
1/1 [==============================] - 0s 194ms/step - loss: 0.6730 - accuracy: 0.5156
Epoch 6/15
1/1 [==============================] - ETA: 0s - loss: 0.6543 - accuracy: 1.0000
 Reached 99.9% accuracy, stopping...
1/1 [==============================] - 0s 78ms/step - loss: 0.6543 - accuracy: 1.0000


1.0